In [171]:
import datetime
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
import scipy.stats as stats
from scipy.stats import f 

from statsmodels.stats.multicomp import pairwise_tukeyhsd
import matplotlib.pyplot as plt
import math
import statsmodels.api as sm
import statistics as st
import os.path
import seaborn as sns
from math import sqrt
import statsmodels.stats.api as sms


Ejercicio 1

In [172]:
df_ej_1 = pd.read_csv('./Ejercicio1.csv')

# Esto es la muestra de la que disponemos
productos = list(df_ej_1['0'])

# Esta es la media que nos dicen que da la población de máquinas usadas
media = 500


In [173]:
stat, p_valor=stats.ttest_1samp(a = productos,
                 popmean = media)

# El p_valor es superior a 0.05. Por tanto la muestra tiene misma media que la población
# No hace falta ajustar las máquinas
p_valor


0.12915779464487212

In [174]:
productos_inf_por = (sum([producto for producto in productos if producto < 500]) / sum(productos)) * 100

productos_inf_por
# Un 39.74% de productos de la muestra está por debajo de la media

39.740023491759196

Ejercicio 2

In [175]:
df_ej_2 = pd.read_csv('./Ejercicio_testHipotesis.csv')


In [176]:
def index_by_datetime(dataframe, name='date'):
    '''Sets a datetime column as an index'''
    '''Requires a dataframe with a datetime column as a positional argument and it may have an alternative name (string) for that column as a key argument'''
    '''Requires pandas as pd'''
    dataframe.sort_values(name, inplace=True)
    dataframe[name] = pd.to_datetime(dataframe[name], exact=False)
    dataframe.set_index(name, inplace=True)


In [177]:
index_by_datetime(df_ej_2, name='fecha')
df_ej_2.drop(columns='Unnamed: 0', inplace=True)


In [178]:
# Si comparamos las máquinas juntas para el total de años, las medias son diferentes
stats.chi2_contingency(df_ej_2)[1]


0.0

In [179]:
inicio_2018 = datetime.datetime(2018, 1, 1, 0, 0, 0)
inicio_2019 = datetime.datetime(2019, 1, 1, 0, 0, 0)

df_ej_2_2017 = df_ej_2[df_ej_2.index < inicio_2018]
df_ej_2_2018 = df_ej_2[(df_ej_2.index >= inicio_2018) & (df_ej_2.index < inicio_2019)]
df_ej_2_2019 = df_ej_2[(df_ej_2.index >= inicio_2019)]

dfs_years = [df_ej_2_2017, df_ej_2_2018, df_ej_2_2019]


In [180]:
# Si comparamos las máquinas juntas para cada uno de los tres años, las medias son diferentes
for df in dfs_years:
    print(df.index[0].year, stats.chi2_contingency(df)[1])


2017 0.0
2018 0.0
2019 0.0


In [181]:
def calcula_contingencia_mensual(dfs):
    '''Requiere una tupla de dataframes'''
    for df in dfs:
        meses_fin = [df.index[-1] + relativedelta(month=mes) for mes in range(1, 13)]
        for index, mes in enumerate(meses_fin):
            if index == 0:
                df_month = df[df.index <= mes]
            else:
                df_month = df[(df.index <= mes) & (df.index > meses_fin[index - 1])]
            print(f"{mes.year} {mes.month}: {stats.chi2_contingency(df_month)[1]}")


In [182]:
calcula_contingencia_mensual(dfs_years)
# Mes a mes las medias también son diferentes si comparamos las máquinas juntas


2017 1: 0.0
2017 2: 0.0
2017 3: 0.0
2017 4: 0.0
2017 5: 0.0
2017 6: 0.0
2017 7: 0.0
2017 8: 0.0
2017 9: 0.0
2017 10: 0.0
2017 11: 0.0
2017 12: 0.0
2018 1: 0.0
2018 2: 0.0
2018 3: 0.0
2018 4: 0.0
2018 5: 0.0
2018 6: 0.0
2018 7: 0.0
2018 8: 0.0
2018 9: 0.0
2018 10: 0.0
2018 11: 0.0
2018 12: 0.0
2019 1: 0.0
2019 2: 0.0
2019 3: 0.0
2019 4: 0.0
2019 5: 0.0
2019 6: 0.0
2019 7: 0.0
2019 8: 0.0
2019 9: 0.0
2019 10: 0.0
2019 11: 0.0
2019 12: 0.0


In [183]:
'''Calculates ftest (are both variances equal?) and returns its pvalue'''
def Ftest(group_1,group_2):
    '''Requires two lists as positional arguments'''
    '''Requires numpy as np and f from scipy.stats'''
    a1 = 1.0*np.array(group_1)
    n1 = len(a1)
    a2 = 1.0*np.array(group_2)
    n2 = len(a2)
    s1, s2 = np.std(a1,ddof=1), np.std(a2,ddof=1)
    test=(s1/s2)**2
    p_value=2*min(f.cdf(test,n1-1,n2-1),1-f.cdf(test,n1-1,n2-1))
    return p_value


In [184]:
def compara_medias_columnas(dataframe):
    '''Requiere un dataframe como argumento posicional'''
    columnas_hechas = []
    lista_comparaciones = []
    for column_1 in dataframe:
        for column_2 in dataframe:
            igualdad = f"Las medias de {column_1} y {column_2} son iguales (pvalor="
            superioridad = f"La media de {column_1} es superior a la de {column_2} (pvalor="
            pvalor_ttest = "Pendiente de calcular"
            if column_1 != column_2 and (column_1, column_2) not in columnas_hechas:
                columnas_hechas.append((column_2, column_1))
                ftest = Ftest(dataframe[column_1], dataframe[column_2])
                if ftest > 0.05:
                    if len(dataframe[column_1]) > 30:
                        pvalor = sm.stats.ztest(dataframe[column_1], dataframe[column_2])[1]
                        if pvalor > 0.05:
                            lista_comparaciones.append(igualdad + str(pvalor) + ")")
                        else:
                            pvalor = sm.stats.ztest(dataframe[column_1], dataframe[column_2], alternative='larger')[1]
                            if pvalor < 0.05:
                                lista_comparaciones.append(superioridad + str(pvalor) + ")")
                    else:
                        pvalor_ttest = stats.ttest_ind(dataframe[column_1], dataframe[column_2], equal_var=True)[1]
                else:
                    pvalor_ttest = stats.ttest_ind(dataframe[column_1], dataframe[column_2], equal_var=False)[1]
                if pvalor_ttest != "Pendiente de calcular":
                    if pvalor_ttest > 0.05:
                        lista_comparaciones.append(igualdad + str(pvalor_ttest) + ")")
                    else:
                        pvalor_ttest = stats.ttest_ind(dataframe[column_1], dataframe[column_2], equal_var=False, alternative='greater')[1]
                        if pvalor_ttest < 0.05:
                            lista_comparaciones.append(superioridad + str(pvalor_ttest) + ")")
    return lista_comparaciones


In [185]:
for df in dfs_years:
    lista_comparaciones = compara_medias_columnas(df)
    lista_superioridades = [comparacion for comparacion in lista_comparaciones if "son iguales" not in comparacion]
    print(f'\nPARA EL AÑO {df.index[0].year}')
    for superioridad in lista_superioridades:
        print(superioridad)

# Comparadas año a año, estas son las diferencias entre máquinas


PARA EL AÑO 2017
La media de C es superior a la de D (pvalor=0.007559073552382409)
La media de C es superior a la de G (pvalor=0.005305341307181211)
La media de C es superior a la de H (pvalor=0.0016374006267286397)
La media de C es superior a la de I (pvalor=0.006847862835224958)
La media de C es superior a la de J (pvalor=0.015893966700395806)
La media de C es superior a la de K (pvalor=0.0014857697353888135)
La media de F es superior a la de G (pvalor=0.012766978916390292)
La media de F es superior a la de H (pvalor=0.0044585446624601476)
La media de F es superior a la de I (pvalor=0.015966092425728266)
La media de F es superior a la de K (pvalor=0.004069942521303673)

PARA EL AÑO 2018

PARA EL AÑO 2019
La media de A es superior a la de D (pvalor=0.008585324745209151)
La media de A es superior a la de J (pvalor=0.021612178041276357)
La media de C es superior a la de D (pvalor=0.011279872358067704)
La media de F es superior a la de J (pvalor=0.020659755032746113)
La media de G es su

In [186]:
for df in dfs_years:
    meses_fin = [df.index[-1] + relativedelta(month=mes) for mes in range(1, 13)]
    for index, mes in enumerate(meses_fin):
        if index == 0:
            df_month = df[df.index <= mes]
        else:
            df_month = df[(df.index <= mes) & (df.index > meses_fin[index - 1])]
            lista_comparaciones = compara_medias_columnas(df_month)
            lista_superioridades = [comparacion for comparacion in lista_comparaciones if "son iguales" not in comparacion]
            print(f'\nPARA EL MES {int(df_month.index[0].month) - 1} DEL {df_month.index[0].year}:')
            for superioridad in lista_superioridades:
                print(superioridad)
# Comparadas mes a mes, estas son las diferencias entre máquinas


PARA EL MES 1 DEL 2017:

PARA EL MES 2 DEL 2017:

PARA EL MES 3 DEL 2017:
La media de C es superior a la de D (pvalor=0.021407343082363976)

PARA EL MES 4 DEL 2017:
La media de C es superior a la de I (pvalor=0.022562961078818826)
La media de C es superior a la de J (pvalor=0.01580615062105758)
La media de C es superior a la de K (pvalor=0.005881545475974373)
La media de G es superior a la de K (pvalor=0.009272815825559541)

PARA EL MES 5 DEL 2017:
La media de J es superior a la de K (pvalor=0.005830382643554065)

PARA EL MES 6 DEL 2017:

PARA EL MES 7 DEL 2017:

PARA EL MES 8 DEL 2017:
La media de A es superior a la de B (pvalor=0.018017414383588455)
La media de A es superior a la de D (pvalor=0.003839967047885111)
La media de A es superior a la de G (pvalor=0.0060061473092935)
La media de A es superior a la de H (pvalor=0.015253579025420962)
La media de A es superior a la de I (pvalor=0.019568579186945965)
La media de F es superior a la de G (pvalor=0.010258012911077858)
La media de

Ejercicio 3

In [187]:
df_ej_3_2020 = pd.read_csv('./Ejercicio_testHipotesis_test.csv')


In [188]:
index_by_datetime(df_ej_3_2020, name='fecha')
df_ej_3_2020.drop(columns='Unnamed: 0', inplace=True)


In [195]:
df_ej_3_2020

,A,B,C,D,F,G,H,I,J,K
fecha,,,,,,,,,,
2020-01-01,1272.854060,2046.847960,971.521946,1079.989248,1092.139214,686.295566,985.477934,863.879412,922.132190,786.337095
2020-01-02,1402.022562,1589.567360,1016.796724,1527.251588,1161.323860,1013.842420,1050.163942,899.780632,744.553762,842.634362
2020-01-03,1214.871224,986.705401,1579.418649,1305.829335,887.841074,774.863555,755.353085,866.139872,883.004593,955.418965
2020-01-04,1557.054440,1328.048373,885.111093,990.861988,1224.485975,727.981625,819.712864,895.324421,963.931586,1152.813584
2020-01-05,792.993228,1070.527530,1544.639097,1404.627023,1541.640922,892.097503,790.684805,820.221737,789.029336,1042.345568
...,...,...,...,...,...,...,...,...,...,...
2020-12-26,889.629740,1542.213560,1655.660145,1037.774096,1142.715482,887.956472,922.069035,1046.379779,685.321389,972.425385
2020-12-27,1188.038200,1236.343682,1571.162859,1275.692457,1405.549701,1141.791630,929.408477,894.511836,838.356960,854.124938
2020-12-28,1220.398143,1061.361077,1465.609626,1136.391362,666.276402,932.944953,805.963442,678.464671,1019.218006,797.710455


In [197]:
df_ej_3 = pd.concat([df_ej_2, df_ej_3_2020], axis=0)

In [201]:
# Si comparamos las máquinas juntas para el total de años, las medias son diferentes
stats.chi2_contingency(df_ej_3)[1]


0.0

In [203]:
inicio_2018 = datetime.datetime(2018, 1, 1, 0, 0, 0)
inicio_2019 = datetime.datetime(2019, 1, 1, 0, 0, 0)
inicio_2020 = datetime.datetime(2020, 1, 1, 0, 0, 0)

df_ej_3_2017 = df_ej_3[df_ej_3.index < inicio_2018]
df_ej_3_2018 = df_ej_3[(df_ej_3.index >= inicio_2018) & (df_ej_3.index < inicio_2019)]
df_ej_3_2019 = df_ej_3[(df_ej_3.index >= inicio_2019) & (df_ej_3.index < inicio_2020)]
df_ej_3_2020 = df_ej_3[(df_ej_3.index >= inicio_2020)]

dfs_years = [df_ej_2_2017, df_ej_2_2018, df_ej_2_2019, df_ej_3_2020]


In [204]:
# Si comparamos las máquinas juntas para cada uno de los cuatro años, las medias son diferentes
for df in dfs_years:
    print(df.index[0].year, stats.chi2_contingency(df)[1])


2017 0.0
2018 0.0
2019 0.0
2020 0.0


In [205]:
calcula_contingencia_mensual(dfs_years)
# Mes a mes las medias también son diferentes si comparamos las máquinas juntas


2017 1: 0.0
2017 2: 0.0
2017 3: 0.0
2017 4: 0.0
2017 5: 0.0
2017 6: 0.0
2017 7: 0.0
2017 8: 0.0
2017 9: 0.0
2017 10: 0.0
2017 11: 0.0
2017 12: 0.0
2018 1: 0.0
2018 2: 0.0
2018 3: 0.0
2018 4: 0.0
2018 5: 0.0
2018 6: 0.0
2018 7: 0.0
2018 8: 0.0
2018 9: 0.0
2018 10: 0.0
2018 11: 0.0
2018 12: 0.0
2019 1: 0.0
2019 2: 0.0
2019 3: 0.0
2019 4: 0.0
2019 5: 0.0
2019 6: 0.0
2019 7: 0.0
2019 8: 0.0
2019 9: 0.0
2019 10: 0.0
2019 11: 0.0
2019 12: 0.0
2020 1: 0.0
2020 2: 0.0
2020 3: 0.0
2020 4: 0.0
2020 5: 0.0
2020 6: 0.0
2020 7: 0.0
2020 8: 0.0
2020 9: 0.0
2020 10: 0.0
2020 11: 0.0
2020 12: 0.0


In [206]:
for df in dfs_years:
    lista_comparaciones = compara_medias_columnas(df)
    lista_superioridades = [comparacion for comparacion in lista_comparaciones if "son iguales" not in comparacion]
    print(f'\nPARA EL AÑO {df.index[0].year}')
    for superioridad in lista_superioridades:
        print(superioridad)

# Comparadas año a año, estas son las diferencias entre máquinas


PARA EL AÑO 2017
La media de C es superior a la de D (pvalor=0.007559073552382409)
La media de C es superior a la de G (pvalor=0.005305341307181211)
La media de C es superior a la de H (pvalor=0.0016374006267286397)
La media de C es superior a la de I (pvalor=0.006847862835224958)
La media de C es superior a la de J (pvalor=0.015893966700395806)
La media de C es superior a la de K (pvalor=0.0014857697353888135)
La media de F es superior a la de G (pvalor=0.012766978916390292)
La media de F es superior a la de H (pvalor=0.0044585446624601476)
La media de F es superior a la de I (pvalor=0.015966092425728266)
La media de F es superior a la de K (pvalor=0.004069942521303673)

PARA EL AÑO 2018

PARA EL AÑO 2019
La media de A es superior a la de D (pvalor=0.008585324745209151)
La media de A es superior a la de J (pvalor=0.021612178041276357)
La media de C es superior a la de D (pvalor=0.011279872358067704)
La media de F es superior a la de J (pvalor=0.020659755032746113)
La media de G es su

In [207]:
for df in dfs_years:
    meses_fin = [df.index[-1] + relativedelta(month=mes) for mes in range(1, 13)]
    for index, mes in enumerate(meses_fin):
        if index == 0:
            df_month = df[df.index <= mes]
        else:
            df_month = df[(df.index <= mes) & (df.index > meses_fin[index - 1])]
            lista_comparaciones = compara_medias_columnas(df_month)
            lista_superioridades = [comparacion for comparacion in lista_comparaciones if "son iguales" not in comparacion]
            print(f'\nPARA EL MES {int(df_month.index[0].month) - 1} DEL {df_month.index[0].year}:')
            for superioridad in lista_superioridades:
                print(superioridad)
# Comparadas mes a mes, estas son las diferencias entre máquinas



PARA EL MES 1 DEL 2017:

PARA EL MES 2 DEL 2017:

PARA EL MES 3 DEL 2017:
La media de C es superior a la de D (pvalor=0.021407343082363976)

PARA EL MES 4 DEL 2017:
La media de C es superior a la de I (pvalor=0.022562961078818826)
La media de C es superior a la de J (pvalor=0.01580615062105758)
La media de C es superior a la de K (pvalor=0.005881545475974373)
La media de G es superior a la de K (pvalor=0.009272815825559541)

PARA EL MES 5 DEL 2017:
La media de J es superior a la de K (pvalor=0.005830382643554065)

PARA EL MES 6 DEL 2017:

PARA EL MES 7 DEL 2017:

PARA EL MES 8 DEL 2017:
La media de A es superior a la de B (pvalor=0.018017414383588455)
La media de A es superior a la de D (pvalor=0.003839967047885111)
La media de A es superior a la de G (pvalor=0.0060061473092935)
La media de A es superior a la de H (pvalor=0.015253579025420962)
La media de A es superior a la de I (pvalor=0.019568579186945965)
La media de F es superior a la de G (pvalor=0.010258012911077858)
La media de